In [1]:
import findspark

In [3]:
findspark.init('/home/mukund/spark-2.1.0-bin-hadoop2.7/')

In [4]:
import pyspark

In [7]:
from pyspark.sql import SparkSession

In [9]:
Spark=SparkSession.builder.appName('lreg').getOrCreate()

In [10]:
from pyspark.ml.regression import LinearRegression

In [53]:
#IMPLEMENTING SIMPLE LINEAR REGRESSION WITHOUT TRAIN-TEST SPLIT, JUST TO GET A HANG OF THE WORKFLOW

In [54]:
training=Spark.read.format('libsvm').load('/home/mukund/ml/sample_linear_regression_data.txt')

In [55]:
training.show()
#spark requires to have dataset for machine learning in this format i.e. two columns, "label" column
#that has either class label or regression value, and "features" column which is a feature vector of all the
#features for that particular observation

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [56]:
lr=LinearRegression(featuresCol='features',labelCol='label')

In [57]:
train=lr.fit(training)

In [58]:
#attributes of trained model
train.intercept

0.14228558260358093

In [59]:
train.coefficients.values

array([ 0.00733507,  0.83137576, -0.8095308 ,  2.44119169,  0.51917138,
        1.15345919, -0.29891241, -0.51285142, -0.61971283,  0.69561518])

In [60]:
train.summary.meanAbsoluteError #accessing attribute that contains error metrics

8.145215527783876

In [61]:
train.summary.rootMeanSquaredError #accessing attribute that contains error metrics

10.16309157133015

In [62]:
train.summary.r2 #accessing attribute that contains error metrics

0.027839179518600154

In [63]:
#IMPLEMENTING SIMPLE LINEAR REGRESSION WITH TRAIN-TEST SPLIT

In [67]:
super_set=Spark.read.format('libsvm').load('/home/mukund/ml/sample_linear_regression_data.txt')

In [68]:
train_data,test_data=super_set.randomSplit([0.8,0.2])

In [70]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                401|
|   mean|0.09471952470024099|
| stddev| 10.293955188742148|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [71]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                100|
|   mean| 0.9071877051489251|
| stddev| 10.439980140337028|
|    min|-26.805483428483072|
|    max| 27.111027963108548|
+-------+-------------------+



In [72]:
model=lr.fit(train_data)

In [78]:
result=model.evaluate(test_data) #evaluating the model on test data

In [79]:
result.rootMeanSquaredError

10.320469333598604

In [81]:
#Model deployement occurs on unlabeled data. Let us create an unlabeled set.
unlabled1,unlabled2=super_set.randomSplit([0.5,0.5])

In [84]:
unlabled=unlabled1.select('features')

In [85]:
unlabled.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [86]:
#now let us deploy our trained model on this unlabled data
prediction=model.transform(unlabled)

In [87]:
prediction.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-1.6039973172558066|
|(10,[0,1,2,3,4,5,...| 1.1521283915487144|
|(10,[0,1,2,3,4,5,...|  2.921903169988495|
|(10,[0,1,2,3,4,5,...|-3.9640757473726507|
|(10,[0,1,2,3,4,5,...|0.11404369850705515|
|(10,[0,1,2,3,4,5,...|0.30182866560647015|
|(10,[0,1,2,3,4,5,...| 0.6303253659018874|
|(10,[0,1,2,3,4,5,...|0.10713573760430106|
|(10,[0,1,2,3,4,5,...| 0.1441863751681527|
|(10,[0,1,2,3,4,5,...|0.32713798706323505|
|(10,[0,1,2,3,4,5,...|-3.0731624885301017|
|(10,[0,1,2,3,4,5,...|-1.5032564281974305|
|(10,[0,1,2,3,4,5,...|-2.2578527076225257|
|(10,[0,1,2,3,4,5,...|-1.0268082463274688|
|(10,[0,1,2,3,4,5,...| 1.0361912355173153|
|(10,[0,1,2,3,4,5,...|-1.3792241592679315|
|(10,[0,1,2,3,4,5,...|-0.6809953662911677|
|(10,[0,1,2,3,4,5,...| 1.3571935402364712|
|(10,[0,1,2,3,4,5,...| 0.2340511338404706|
|(10,[0,1,2,3,4,5,...|-0.7933171757633518|
+----------